Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/content/gdrive/My Drive/Project


In [3]:
# !pip install mxnet
# !pip install gluonnlp pandas tqdm
# !pip install sentencepiece
# !pip install transformers==3.0.2
# !pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-3.0.2-py3-none-any.whl (769 kB)
  Using cached tokenizers-0.8.1rc1-cp37-cp37m-manylinux1_x86_64.whl (3.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.10.3
    Uninstalling tokenizers-0.10.3:
      Successfully uninstalled tokenizers-0.10.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.8.1
    Uninstalling transformers-4.8.1:
      Successfully uninstalled transformers-4.8.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviou

In [4]:
# !pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-3zq0hf30
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-3zq0hf30
  Using cached transformers-4.8.1-py3-none-any.whl (2.5 MB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
  Attempting uninstall: transformers
    Found existing installation: transformers 3.0.2
    Uninstalling transformers-3.0.2:
      Successfully uninstalled transformers-3.0.2


In [1]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [2]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [3]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model. /home/jupyter-j7d203/Untitled Folder/.cache/kobert_v1.zip
using cached model. /home/jupyter-j7d203/Untitled Folder/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [4]:
tokenizer = get_tokenizer()

using cached model. /home/jupyter-j7d203/Untitled Folder/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [5]:
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

In [6]:
max_len = 64
batch_size = 128
warmup_ratio = 0.1
num_epochs = 3
max_grad_norm = 1
log_interval = 200
learning_rate = 5e-5


In [7]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=5,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [8]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
    transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx])for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i],))

  def __len__(self):
    return(len(self.labels))

In [11]:
model = BERTClassifier(bertmodel,  dr_rate=0.5)
model.load_state_dict(torch.load('./model6.h5'))
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [12]:
def predict(predict_sentence):
  
  data = [predict_sentence, '0']
  dataset_another = [data]

  another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
  test_dataloader = torch.utils.data.DataLoader(another_test, batch_size = batch_size, num_workers=5)

  model.eval()

  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
    token_ids = token_ids.long()
    segment_ids = segment_ids.long()

    valid_length = valid_length
    label = label.long()

    out = model(token_ids, valid_length, segment_ids)

    test_eval=[]

    for i in out:
      logits = i
      logits = logits.detach().numpy()
    
      if np.argmax(logits) == 0:
        test_eval.append("분노")
      elif np.argmax(logits) == 1:
        test_eval.append("놀람")
      elif np.argmax(logits) == 2:
        test_eval.append("불안")
      elif np.argmax(logits) == 3:
        test_eval.append("행복")
      elif np.argmax(logits) == 4:
        test_eval.append("슬픔")
      # elif np.argmax(logits) == 5:
      #   test_eval.append("당황이")
   
#     print(">> 입력하신 내용에서 "+ test_eval[0] + " 느껴집니다.")
#     print(min(logits))
#     print(logits)
    min_logits = min(logits)
    if min_logits < 0:
        for q in range(len(logits)):
            logits[q]+=abs(min_logits)
    total = sum(logits)
    emotions_scores =[]
    for w in range(len(logits)):
        emotions_scores.append(round(((logits[w]/total) * 100), 2))
#     print(emotions_scores)
#     print(sum(emotions_scores))
    return emotions_scores, test_eval[0]

In [18]:
emotions_scores, emo =predict("친구집에서 자기로 했는데 친구가 갑자기 안 된다고했다.")
print(emotions_scores)
print(emo)

[24.88, 6.09, 26.39, 0.0, 42.63]
슬픔
